In [ ]:
## Duplicate of finding_award_winners

In [1]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from collections import defaultdict
import en_core_web_md
nlp = en_core_web_md.load()

In [2]:
award_names = ['Best Motion Picture – Drama', 'Best Motion Picture – Musical or Comedy', 'Best Motion Picture – Foreign Language',
 'Best Motion Picture – Animated', 'Best Director – Motion Picture', 'Best Actor in a Motion Picture – Drama',
 'Best Actor in a Motion Picture – Musical or Comedy', 'Best Actress in a Motion Picture – Drama',
 'Best Actress in a Motion Picture – Musical or Comedy', 'Best Supporting Actor – Motion Picture',
 'Best Supporting Actress – Motion Picture', 'Best Screenplay – Motion Picture', 'Best Original Score – Motion Picture',
 'Best Original Song – Motion Picture', 'Best Television Series – Drama', 'Best Television Series – Musical or Comedy',
 'Best Miniseries or Motion Picture – Television', 'Best Actor in a Television Series – Drama',
 'Best Actor in a Television Series – Musical or Comedy', 'Best Actor in a Miniseries or Motion Picture – Television',
 'Best Actress in a Television Series – Drama', 'Best Actress in a Television Series – Musical or Comedy',
 'Best Actress in a Miniseries or Motion Picture – Television',
 'Best Supporting Actor in a Series, Miniseries or Motion Picture – Television',
 'Best Supporting Actress in a Series, Miniseries or Motion Picture – Television']

new_award_names = [0]*len(award_names)
for i in range(len(award_names)):
#     new_award_names[i] = award_names[i].lower().
    new_award_names[i] = award_names[i].replace(" in ", " ").replace(" a ", " ").replace(" or ", " ").replace(" – ", " ").replace(" - ", " ").lower()
#     if "television" in new_award_names[i]:
#         new_award_names[i].append("tv")
new_award_names

['best motion picture drama',
 'best motion picture musical comedy',
 'best motion picture foreign language',
 'best motion picture animated',
 'best director motion picture',
 'best actor motion picture drama',
 'best actor motion picture musical comedy',
 'best actress motion picture drama',
 'best actress motion picture musical comedy',
 'best supporting actor motion picture',
 'best supporting actress motion picture',
 'best screenplay motion picture',
 'best original score motion picture',
 'best original song motion picture',
 'best television series drama',
 'best television series musical comedy',
 'best miniseries motion picture television',
 'best actor television series drama',
 'best actor television series musical comedy',
 'best actor miniseries motion picture television',
 'best actress television series drama',
 'best actress television series musical comedy',
 'best actress miniseries motion picture television',
 'best supporting actor series, miniseries motion picture

In [11]:
awardnominees = {}
awardnominees = defaultdict(lambda: 1, awardnominees)

for award in award_names:
    awardnominees[award] = ['winner']

df = pd.read_json('../gg2013.json')
df = df['text']
df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)
# nominees_df = df[df.str.contains('nominated', case = False)]
nominees_df = df[df.str.contains('Best', case = True)]
# nominees_df = nominees_df[~nominees_df.str.contains('win|RT', case = False)]
# nominees_df = nominees_df[nominees_df.str.contains('nominated', case = False)]
# df = df[~df.str.contains('think|thinking|should|maybe', case = False)]

if nominees_df.shape[0] > 4000:
    nominees_df = nominees_df.sample(n=4000)
nominees_df.shape[0]

print(nominees_df.head(100))
print(len(nominees_df))

<ipython-input-11-125592c12ac6>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.str.replace('#GoldenGlobes|golden|globes|globe', "", case = False)


121806                           Jodie Foster.  #BestSpeech
99784     RT @: Best Original Song - "Skyfall" - Adele (...
85512     RT @: Best Screenplay - Quentin Tarantino - Dj...
81694     RT @: Best Supporting Actress in a Motion Pict...
14304     RT @THR: Christoph Waltz wins the   for Best S...
                                ...                        
71746     RT @TheThryll: When is the "Best Storage Wars ...
128824                    Best Director Ben Affleck - Argo 
45156      Best Original Song - Motion Picture goes to S...
97091          RT @: Best Foreign Film - Amour (Austria) - 
110037    RT @girlsHBO: A massive congratulations to the...
Name: text, Length: 100, dtype: object
4000


In [4]:
def closeenough(a,b): 
  a = a.split() 
  b = b.split() 
  k = set(a).symmetric_difference(set(b)) 
  return True if len(k) <= 2 else False

In [5]:
from spacy.matcher import Matcher
from spacy.tokens import Span
from string import punctuation
import operator

In [6]:
pattern1 = [{"ORTH":'Best'}, {"DEP": 'compound', 'OP':'+'}, {"POS":'NOUN', 'OP':'*'}, {'IS_PUNCT': True, 'OP':'?'}, {"POS": 'PROPN', 'OP':'*'}]
pattern2 = [{"ORTH":'Best'}, {"ENT_TYPE": "WORK_OF_ART", 'OP':'+'}]


In [7]:
matcher = Matcher(nlp.vocab)
matcher.add("Award", [pattern1])

In [8]:
real = {}
real = defaultdict(lambda: 1, real)
maybe = {}
maybe = defaultdict(lambda: 1, maybe)
maybenot = {}
maybenot = defaultdict(lambda: 1, maybenot)
for i, text in nominees_df.iteritems():
    doc = nlp(text)
    matches = matcher(doc)
    if len(matches) != 0:
        span = doc[matches[-1][1]:matches[-1][2]]
        sent = str(span).strip(punctuation).strip().replace('- ','').lower()
        if "television" in sent:
            sent.replace("television",'tv')
            
        awarded = 0
        for i, award in enumerate(new_award_names):
            if sent in award or sent == award or closeenough(sent, award):
                winner = awardnominees[list(awardnominees)[i]][0]
                if winner not in text:
                    for ent in nlp(text.replace("sent","")).ents:
                        if ent.label_ == "PERSON" or ent.label_ == "WORK_OF_ART":
                            awardnominees[list(awardnominees)[i]].append(ent.text)
                            awarded = 1
        
        if awarded == 0:
            maybe[sent] = maybe[sent] + 1
        

In [10]:
new = awardnominees

for lst in new:
    dd = defaultdict( int )
    for word in new[lst]:
        dd[word] += 1
    
    new[lst] = dd

new

defaultdict(<function __main__.<lambda>()>,
            {'Best Motion Picture – Drama': defaultdict(int,
                         {'winner': 1,
                          'Les Mis': 3,
                          'Jennifer Lawrence': 1,
                          'Quinten': 1,
                          'Quentin Tarantino - Django': 1,
                          'Quentin Tarantino': 4,
                          'Unchained': 2,
                          '#tarantino': 1,
                          'Bill Clinton': 2,
                          'Lincoln': 2,
                          'Ari celebrando que #': 1,
                          'Motion Picture - Ben Affleck': 1,
                          'Ben Affleck': 13,
                          'Steven Speilburg': 1,
                          "Bill Murray's": 1,
                          'Argo': 5,
                          'Oscar': 1,
                          'Hugh Jackman Les Miserables': 1,
                          'Dustin Hoffman': 1,
           

In [69]:
doc = nlp("The Best Exotic Marigold Hotel .... um ... what?! never heard of this movie, but its up for Best Motion Picture?! ok! ")
for ent in doc.ents:
    print(ent)

In [22]:
for each in awardnominees:
    print(each)
    print(len(awardnominees[each]))

Best Motion Picture – Drama
55
Best Motion Picture – Musical or Comedy
24
Best Motion Picture – Foreign Language
19
Best Motion Picture – Animated
50
Best Director – Motion Picture
50
Best Actor in a Motion Picture – Drama
45
Best Actor in a Motion Picture – Musical or Comedy
10
Best Actress in a Motion Picture – Drama
46
Best Actress in a Motion Picture – Musical or Comedy
11
Best Supporting Actor – Motion Picture
22
Best Supporting Actress – Motion Picture
22
Best Screenplay – Motion Picture
51
Best Original Score – Motion Picture
34
Best Original Song – Motion Picture
50
Best Television Series – Drama
16
Best Television Series – Musical or Comedy
7
Best Miniseries or Motion Picture – Television
19
Best Actor in a Television Series – Drama
17
Best Actor in a Television Series – Musical or Comedy
4
Best Actor in a Miniseries or Motion Picture – Television
5
Best Actress in a Television Series – Drama
16
Best Actress in a Television Series – Musical or Comedy
5
Best Actress in a Minise

In [13]:
awardnominees

defaultdict(<function __main__.<lambda>()>,
            {'Best Motion Picture – Drama': ['winner',
              'Les Mis',
              'Jennifer Lawrence',
              'Quinten',
              'Quentin Tarantino - Django',
              'Quentin Tarantino',
              'Unchained',
              'Quentin Tarantino',
              'Quentin Tarantino',
              'Unchained',
              '#tarantino',
              'Bill Clinton',
              'Lincoln',
              'Bill Clinton',
              'Lincoln',
              'Ari celebrando que #',
              'Motion Picture - Ben Affleck',
              'Ben Affleck',
              'Ben Affleck',
              'Steven Speilburg',
              'Quentin Tarantino',
              "Bill Murray's",
              'Ben Affleck',
              'Argo',
              'Ben Affleck',
              'Argo',
              'Ben Affleck',
              'Argo',
              'Ben Affleck',
              'Argo',
              'Oscar',
      

In [ ]:
winners = sorted(winners.items(), key=lambda item: item[1], reverse = True)
print(winners)

In [ ]:
import Levenshtein
print(Levenshtein.distance("Hi Bye", "hi bye"))

In [ ]:
award_list = [dict() for x in range(len(new_award_names))]

for tweet in winners: 
    tweet = tweet[0]
    split = tweet.lower().split()
    
    index = split.index("best")

    award_phrase = " ".join(split[index:])

    name_count = [0]*len(new_award_names)
    for i in range(len(new_award_names)):
#         match_length = len(set(new_award_names[i]).intersection(award_phrase))
#         try Levenshtein distance instead, may be more accurate
        
        name_count[i] = Levenshtein.distance(new_award_names[i], award_phrase)

    award_index = np.argmin(name_count)
    # WHAT ABOUT TIE? Neither? Or Both?
    #print(award_index)
    
    print(name_count)

#     if name_count[award_index] > 1:       
    if "wins" in split:
        person_index = split.index("wins")
    if "won" in split:
        person_index = split.index("won")
    person = split[:index-1]
    person = " ".join(person)
    if person not in award_list[award_index]:
        award_list[award_index][person] = 1
    else:
        award_list[award_index][person] = award_list[award_index][person]+1

print(award_list)

In [ ]:
import operator
i = 0
awards = {}
for award in award_list:
    if award != {}:
        awards[i] = max(award.items(), key=operator.itemgetter(1))[0]
    else:
        awards[i] = ''
    i+=1
print(awards)